# AIT 526: Programming Assignment 1 
# Team 2

Download the libraries used

In [2]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg
!python -m spacy download en

  Using cached catalogue-1.0.0-py2.py3-none-any.whl (7.7 kB)
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.6
    Uninstalling catalogue-2.0.6:
      Successfully uninstalled catalogue-2.0.6


You should consider upgrading via the 'c:\users\tewod\anaconda3\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\tewod\anaconda3\python.exe -m pip install --upgrade pip' command.


[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


You should consider upgrading via the 'C:\Users\tewod\anaconda3\python.exe -m pip install --upgrade pip' command.


[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[x] Couldn't link model to 'en'
Creating a symlink in spacy/data failed. Make sure you have the required
permissions and try re-running the command as admin, or use a virtualenv. You
can still import the model as a module and call its load() method, or create the
symlink manually.
C:\Users\tewod\anaconda3\lib\site-packages\en_core_web_sm -->
C:\Users\tewod\anaconda3\lib\site-packages\spacy\data\en
[!] Download successful but linking failed
Creating a shortcut link for 'en' didn't work (maybe you don't have admin
permissions?), but you can still load the model via its full package name: nlp =
spacy.load('en_core_web_sm')


You should consider upgrading via the 'C:\Users\tewod\anaconda3\python.exe -m pip install --upgrade pip' command.
You do not have sufficient privilege to perform this operation.


Import Libraries

In [36]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import time
import re
import random 
import spacy

Set the initial interface for Eliza and the users

In [37]:
eliza_inter = "[eliza]: {0}"
user_inter = "[{0}]: "

This part is about identifying users name from the words the customer entered. If it included person name, it is recogized as a name. And if the users only typed one word, it is recognized as a name. If the user typed the uppercase words, then the word is recogized as a name, otherwise just ignore it.

In [38]:
def name_user(user_typing):
    en = spacy.load('en_core_web_sm')
    doc = en(user_typing)
    
    name_list = [i for i in doc.ents if i.label_ == "person"]

    if name_list:
        user_name = name_list[0].text
        eliza_response = "It is good to meet you, " + user_name +". How are you feeling today ?"
    else:

        name_token = word_tokenize(user_typing)
        if len(name_token) == 1:
           
            user_name = name_token[0]
            eliza_response = "It is good to meet you, " + user_name +". How are you feeling today ?"
        else:
            name_words = [w for w in name_token if re.search(r'[A-Z][A-z][a-z]', w)]
            user_name = name_words
            if len(name_words) == 0:
                user_name = ""
                eliza_response = "It is good to meet you, how are you feeling today ?"
            else:
                user_name = name_words[0]
                eliza_response = "It is good to meet you, " + user_name +". How are you feeling today ?"
        print(eliza_inter.format(eliza_response))
        return user_name           

The following rules, known as keys, have been built with the aid of the keywords from the questions, and the values of the key are the possible responses.

In [39]:
rules = {r'I need some (.*).':["Would it really help you to get some {0} ?", "Would that help you?"],
         r'How are you?':[ "I'm fine, thank you. And you?", "I am good, how about you?"],
         r'I cannot (.*).':[" Why you can't {0} ? ", "Perhaps you could try {0} if you want.","Can I know the reason?"],
         r'I am (.*).':[ "Did you come to me because you are {0}?","How do you feel about being {0}"],
         r'(.*)(good|happy|excited)(.)*':["That's great. Keep rocking!","I am happy you are feeling that"],
         r'(.*)(sad|anxious|depressed|angry|stressed)(.)*':[ " Why do you feel this way?"," I am sorry, how can I make you feel better?"," What do you do to make yourself feel better?"," Do you want to tell me more?"],
         r'Yes,(.)*':["Okay."],
         r'No,(.)*':[" Are you sure?", "Okay, tell me more" ],
         r'(.*)(cook.ing|dance.ing|sleep.ing|read.ing|walk.ing|music).':[" Yes, that is something you could continue doing to cheer yourself up!", " I am glad you are talking to me. I would like to know more about your family and friends."],
         r'(.*)family(.)*':[ " Have you told your family about how you are feeling?", " Tell me more about your family."          ],
         r'Are you (.*)':["Why does it matter whether I am {0}?", "Would you prefer that I were not {0}", "Maybe you believe I am {0}", "I may be {0} -- what do you think?"],
         r'What (.*)':["Why do you ask {0}?", "How would an answer to that question help {0}?", "What do you think {0}?"],
         r'Because (.*)':["Is that the reason?", "What other reasons come to mind?", "Does that reason apply to anything else?", "If {0}. what else must be true?"],
         r'(.*) sorry (.*)':["There are many times when no apology is needed.", "What feelings do you have when you apologize?"],
         r'Hello(.*)':["Hello... I am glad you could drop by today.", "Hi there... How are you doing today?", "Hello, How is it going today?"],
         r'You think (.*)':["Do you doubt {0}?", "Do you really think so?", "But you are not 100% sure {0}?"],
         r'Can you (.*)':["What makes you think I can't {0}?", "If I could {0}, then what?", "Why do you ask if I can {0}?"],
         r'You are (.*)':["Why do you say I am {0}?", "Why do you think I am {0}?", "Are we talking about you or me?"],
         r'I do not (.*)':["Don't you really {0}", "Why don't you {0}", "Do you want to {0}?"],
         r'I feel (.*)':["Good, tell me more about those feelings.", "Do you often feel {0}?", "When do you usually feel {0}?", "When you feel {0}?"],
         r'What should I do to be more (.*)?':["To be more {0} , you can socialize, try to excercise, and do the things that make you happy."],
         r'(.*)':["So, how did you sleep?", "Life is messy, chaotic, confusing and that’s okay.","Some things you’re stressing about just aren’t worth it.", "Be nice to yourself, you’re doing your best.","I don't understand", "Can you tell me more?", "Okay, what do you think about that?"]}

Define how the responses can match the rules. If the response from the users can match any key of the dictionary, just randomly choose one answer from the values, and if the answer has {0}, we can use the word at (.*) to change it.

In [40]:
def match(rules,user_typing):
    eliza_reply, u_says = " ", None
    for key,value in rules.items():
        match_rules = re.search(key, user_typing)
        if match_rules is not None:
            eliza_reply = random.choice(value)
            for word in eliza_reply:
                if '{0}' in eliza_reply:
                    u_says = match_rules.group(1)
                    return eliza_reply.format(u_says)
                else:
                    eliza_reply = random.choice(value)
                    return eliza_reply          

This is the main function in which Eliza can be used. If the user types 'quit', the loop will be break, and the conversation will end.

In [33]:
if __name__ == '__main__':
    print ("Hello, welcome to Eliza! Keep in mind that this is a safe place and I am here to help you. Type 'quit' if you want leave at any point.")
    message = "Hello, I am Eliza and I am a Physchotherapist. What is your name ?"
    print(eliza_inter.format(message))
    
    user_typing = input(user_inter.format("user"))
    user_name = name_user(user_typing)

    while True:
        user_typing = input(user_inter.format(user_name))
        if user_typing == 'quit':
            print("Goodbye, have a nice day!")
            print("....................................")
            break
        
        eliza_response = match(rules,user_typing)
        print(eliza_inter.format(eliza_response))
        
        
print("                                                                                  ")
print("      888888888 8    8      8       88      8 8   8  8      8  888888    8     8  ")
print("          8     8    8     8 8      8 8     8 8  8    8    8  8      8   8     8  ")
print("          8     8    8    8   8     8  8    8 8 8      8  8  8        8  8     8  ")
print("          8     888888   8     8    8   8   8 88        88  8          8 8     8  ")
print("          8     8    8  888888888   8    8  8 8 8       88   8        8  8     8  ")
print("          8     8    8 8         8  8     8 8 8  8      88    8      8   8     8  ")
print("          8     8    88           8 8      88 8   8     88     888888    8888888  ")
        

Hello, welcome to Eliza! Keep in mind that this is a safe place and I am here to help you. Type 'quit' if you want leave at any point.
[eliza]: Hello, I am Eliza and I am a Physchotherapist. What is your name ?
[user]: I am Teddy.
[eliza]: It is good to meet you, Teddy. How are you feeling today ?
[Teddy]: I feed sad.
[eliza]:  What do you do to make yourself feel better?
[Teddy]: Nothing
[eliza]: Be nice to yourself, you’re doing your best.
[Teddy]: I need some advice.
[eliza]: Would that help you?
[Teddy]: Yes.
[eliza]: Can you tell me more?
[Teddy]: What should I do to be more joyful.
[eliza]: Life is messy, chaotic, confusing and that’s okay.
[Teddy]: quit
Goodbye, have a nice day!
....................................
                                                                                  
      888888888 8    8      8       88      8 8   8  8      8  888888    8     8  
          8     8    8     8 8      8 8     8 8  8    8    8  8      8   8     8  
          8     8  




References:

    https://docs.python.org/3/library/re.html
    https://www.masswerk.at/elizabot/